In [295]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import Sequential
from time import time

In [296]:
# Import dataset
train = pd.read_csv('optdigits.tra', header=None)
test = pd.read_csv('optdigits.tes', header=None)

In [297]:
# Sort x and y
train_x = train.iloc[:, 0:64]
train_y = train[64]
test_x = test.iloc[:, 0:64]
test_y = test[64]

In [298]:
# Standardization
train_x = train_x.astype('float32') / 16.
test_x = test_x.astype('float32') / 16.

In [299]:
# Split training set into training set and validation set
new_train_x, val_x, new_train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1)

In [300]:
# Changed dataset into numpy
new_train_x = new_train_x.to_numpy()
new_train_y = new_train_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()

In [301]:
# Change the shape of y data set
new_train_y = to_categorical(new_train_y)
test_y = to_categorical(test_y)
val_y = to_categorical(val_y)

In [302]:
# Creat a model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, activation='tanh'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [303]:
# Set SGD as the optimizer
opt = SGD(learning_rate=0.2, momentum=0.2, nesterov=False)

In [304]:
# Early Stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                              min_delta=0, patience=0, 
                              verbose=0, mode='auto')

In [305]:
# Model Compiling
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [306]:
# Fit Model
start = time()
model.fit(new_train_x, new_train_y, epochs= 180, batch_size=15,
          callbacks = [early_stop], validation_data=(val_x, val_y))
print(time()-start)

Train on 3058 samples, validate on 765 samples
Epoch 1/180
3058/3058 [==============================] - 1s 277us/sample - loss: 0.6018 - accuracy: 0.8623 - val_loss: 0.2258 - val_accuracy: 0.9503
Epoch 2/180
3058/3058 [==============================] - 0s 139us/sample - loss: 0.1856 - accuracy: 0.9545 - val_loss: 0.1653 - val_accuracy: 0.9634
Epoch 3/180
3058/3058 [==============================] - 0s 136us/sample - loss: 0.1360 - accuracy: 0.9630 - val_loss: 0.1387 - val_accuracy: 0.9608
Epoch 4/180
3058/3058 [==============================] - 0s 147us/sample - loss: 0.1126 - accuracy: 0.9696 - val_loss: 0.1335 - val_accuracy: 0.9621
Epoch 5/180
3058/3058 [==============================] - 0s 134us/sample - loss: 0.0957 - accuracy: 0.9755 - val_loss: 0.1222 - val_accuracy: 0.9621
Epoch 6/180
3058/3058 [==============================] - 0s 149us/sample - loss: 0.0863 - accuracy: 0.9801 - val_loss: 0.1176 - val_accuracy: 0.9621
Epoch 7/180
3058/3058 [==============================] - 0s

In [307]:
# Distribution of training set  >>> counted with for loops
dist_train = [376,389,380,389,387,376,377,387,380,382]
# Distribution of testing set
dist_test = [178,182,177,183,181,182,181,179,174,180]

In [308]:
# Accuracay of overall classification accuracy and class accuracy
def accuracy_overall(pred, actual, num_classes):
    result = int()
# Overall classfication accuracy
    overall = np.where(pred == actual)[0]
    overall_acc = float(len(overall))/len(pred)
    result = overall_acc
    return result

def accuracy(pred, actual, num_classes):
# Class accuracy
    result = []
    classes = np.array([float(0)]*10)
    for i in range(0,len(pred)):
        id_classes = int(actual[i])
        if pred[i] == actual[i]:
            classes[id_classes] = classes[id_classes] + 1
    classes_acc = classes/num_classes
    result.append(classes_acc)
    return result

In [309]:
# confusion matrix
def confusion_matrix(pred, actual):
    matrix = np.array([int(0)]*10*10).reshape(10,10)
    for i in range(0,10):
        pred_i = pred[np.where(np.array(actual) == i)]
        conf = []
        for j in range(0,10):
            length = len(np.where(pred_i == j)[0])
            conf.append(length)
        matrix[i,] = conf
    return matrix

In [310]:
test_y = test[64]
test_y = test_y.to_numpy()

In [311]:
predict_train = model.predict(train_x)
predict_train_class = predict_train.argmax(axis=-1)
acc_train_overall = accuracy_overall(predict_train_class, train_y, dist_train)
acc_train_class = accuracy(predict_train_class, train_y, dist_train)
conf_train = confusion_matrix(predict_train_class, train_y)


predict_test = model.predict(test_x)
predict_test_class = predict_test.argmax(axis=-1)
acc_test_overall = accuracy_overall(predict_test_class, test_y, dist_test)
acc_test_class = accuracy(predict_test_class, test_y, dist_test)
conf_test = confusion_matrix(predict_test_class, test_y)

In [312]:
#output results
print("training set overall accuracy" + "\n")
print(acc_train_overall)
print("\n")
print("training set class accuracy" + "\n")
print(acc_train_class)
print("\n")
print("training set confusion matrix" + "\n")
print(conf_train)
print("\n")
print("testing set overall accuracy" + "\n")
print(acc_test_overall)
print("\n")
print("testing set class accuracy" + "\n")
print(acc_test_class)
print("\n")
print("testing set confusion matrix" + "\n")
print(conf_test)

training set overall accuracy

0.9814281977504578


training set class accuracy

[array([0.99468085, 0.98971722, 0.99473684, 0.97686375, 0.95607235,
       0.9787234 , 0.99204244, 0.98449612, 0.98157895, 0.96596859])]


training set confusion matrix

[[374   0   0   0   0   0   1   0   1   0]
 [  0 385   0   0   0   0   0   1   1   2]
 [  1   0 378   0   0   0   0   0   1   0]
 [  0   1   0 380   0   3   0   0   1   4]
 [  1   2   0   0 370   0   3   0   3   8]
 [  0   1   2   1   0 368   1   0   0   3]
 [  0   2   0   0   1   0 374   0   0   0]
 [  0   0   2   2   0   0   0 381   0   2]
 [  0   5   0   0   1   0   1   0 373   0]
 [  1   4   0   2   2   1   0   0   3 369]]


testing set overall accuracy

0.9532554257095158


testing set class accuracy

[array([0.98876404, 0.98901099, 0.96045198, 0.92896175, 0.96685083,
       0.96703297, 0.98342541, 0.87150838, 0.91954023, 0.95555556])]


testing set confusion matrix

[[176   0   1   0   0   1   0   0   0   0]
 [  0 180   0   0   0   0